<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>

# Climate Change Satellite Image Classification Competition Model Submission Guide - keras

---
**About the Original Data:**<br>
*Data and Description accessed from [Tensorflow](https://www.tensorflow.org/datasets/catalog/bigearthnet)* <br>
The BigEarthNet is a new large-scale Sentinel-2 benchmark archive, consisting of 590,326 Sentinel-2 image patches. The image patch size on the ground is 1.2 x 1.2 km with variable image size depending on the channel resolution. This is a multi-label dataset with 43 imbalanced labels, which has been simplified to single labels with 3 categories for the purposes of this competition.

To construct the BigEarthNet, 125 Sentinel-2 tiles acquired between June 2017 and May 2018 over the 10 countries (Austria, Belgium, Finland, Ireland, Kosovo, Lithuania, Luxembourg, Portugal, Serbia, Switzerland) of Europe were initially selected. All the tiles were atmospherically corrected by the Sentinel-2 Level 2A product generation and formatting tool (sen2cor). Then, they were divided into 590,326 non-overlapping image patches. Each image patch was annotated by the multiple land-cover classes (i.e., multi-labels) that were provided from the CORINE Land Cover database of the year 2018 (CLC 2018).

Bands and pixel resolution in meters:

    B01: Coastal aerosol; 60m
    B02: Blue; 10m
    B03: Green; 10m
    B04: Red; 10m
    B05: Vegetation red edge; 20m
    B06: Vegetation red edge; 20m
    B07: Vegetation red edge; 20m
    B08: NIR; 10m
    B09: Water vapor; 60m
    B11: SWIR; 20m
    B12: SWIR; 20m
    B8A: Narrow NIR; 20m

License: Community Data License Agreement - Permissive, Version 1.0."

**Competition Data Specifics:**<br>
For the purpose of this competition, the original BigEarthNet dataset has been simplified to 20,000 images (15,000 training images and 5,000 test images) with 3 categories: "forest", "nonforest", and "snow_shadow_cloud", which contains images of snow and clouds. <br>
Each "image" is a folder with 12 satellite image layers, each of which pics up on different features. The example preprocessor uses just three layers: B02, B03, and B04, which contain the standard RGB layers used in ML models. However, you are free to use any combination of the satellite image layers. 

**Data Source:**<br>
Sumbul, G, Charfuelan, M, Demir, B and Markl, V. (2019). BigEarthNet: A Large-Scale Benchmark Archive For Remote Sensing Image Understanding. *Computing Research Repository (CoRR), abs/1902.06148.* https://www.tensorflow.org/datasets/catalog/bigearthnet




# Overview
---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Load Data

In [1]:
!python -V

Python 3.8.0


In [2]:
#install aimodelshare library
#! pip install aimodelshare-nightly

In [3]:
# Get competition data - May take a couple minutes due to size of data set
# from aimodelshare import download_data
# download_data('public.ecr.aws/y2e2a1d6/climate_competition_data-repository:latest') 

In [4]:
# # Unzip Data - May take a couple minutes due to size of data set
# import zipfile
# with zipfile.ZipFile('climate_competition_data/climate_competition_data.zip', 'r') as zip_ref:
#     zip_ref.extractall('competition_data')

##2.   Preprocess data / Write and Save Preprocessor function


In [5]:
# Set up for data preprocessing
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

import tensorflow_datasets as tfds

In [6]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(imageband_directory):
    """
    This function preprocesses reads in images, resizes them to a fixed shape and
    min/max transforms them before converting feature values to float32 numeric values
    required by onnx files.

    params:
        imageband_directory
            path to folder with 13 satellite image bands

    returns:
        X
            numpy array of preprocessed image data

    """

    import PIL
    import os
    import numpy as np
    import tensorflow_datasets as tfds

    def _load_tif(data):
        """Loads TIF file and returns as float32 numpy array."""
        img = tfds.core.lazy_imports.PIL_Image.open(data)
        img = np.array(img.getdata()).reshape(img.size).astype(np.float32)
        return img

    def preprocess_image(imgarray):
        imgarray = (imgarray - np.mean(imgarray))/np.std(imgarray)
        imgarray = imgarray/np.max(imgarray)
        return imgarray
    
    image_list = [0,0,0]

    filelist1 = os.listdir(imageband_directory)

    for fpath in filelist1:
        fullpath = imageband_directory+"/"+fpath

        if fullpath.endswith('B04.tif'):
            imgarray = _load_tif(imageband_directory+"/"+fpath)
            image_list[0] = imgarray
        
        if fullpath.endswith('B03.tif'):
            imgarray = _load_tif(imageband_directory+"/"+fpath)
            image_list[1] = imgarray
        
        if fullpath.endswith('B02.tif'):
            imgarray = _load_tif(imageband_directory+"/"+fpath)
            image_list[2] = imgarray 
            
    X = np.stack(image_list, axis=2)   # to get (height,width,3)
    
    # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
    X = np.expand_dims(preprocess_image(X), axis=0)

    X = np.array(X, dtype=np.float32)  # Final shape for onnx runtime.
    
    return X

In [7]:
# # Create complete list of file names
# forestfilenames = ["competition_data/trainingdata/forest/" +
#                    x for x in os.listdir("competition_data/trainingdata/forest")]
# nonforestfilenames = ["competition_data/trainingdata/nonforest/" +
#                       x for x in os.listdir("competition_data/trainingdata/nonforest")]
# otherfilenames = ["competition_data/trainingdata/other/" +
#                   x for x in os.listdir("competition_data/trainingdata/other")]

# filenames = forestfilenames + nonforestfilenames + otherfilenames

# # preprocess rbg images into 120,120,3 numpy ndarray
# preprocessed_image_data = []
# for i in filenames:
#     try:
#         preprocessed_image_data.append(preprocessor(i))
#     except Exception as e :
#         print(e)
#         pass

In [8]:
# np.save("XCeption_4thApril",preprocessed_image_data)

In [9]:
preprocessed_image_data = np.load("XCeption_4thApril.npy")

In [22]:
# Set up y data
from itertools import repeat
forest=repeat(0,5000)
nonforest=repeat(1,5000)
other=repeat(2,5000)

# Force Forest to be 0
# Force NonForest to be 1
# Force Other to be 2


ylist=list(forest)+list(nonforest)+list(other)


In [23]:
# Shuffle X and y data
from sklearn.utils import shuffle
X_train, y_train = shuffle(preprocessed_image_data, ylist, random_state=0)

In [24]:
y_train = np.array(y_train)

In [25]:
X_train=np.vstack(X_train) # convert X from list to array

In [26]:
X_train.shape

(15000, 120, 120, 3)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, BatchNormalization, Flatten 

from tensorflow.keras.layers import GlobalAveragePooling2D,AveragePooling2D,Dense
from tensorflow.keras.applications.xception import Xception

from tensorflow.keras.regularizers import L1L2

import tensorflow as tf
import pandas 
import keras.backend as K


In [74]:
# Run using GPU
with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
    model = Sequential()

    
    model.add(Xception(include_top=False , input_shape=(120,120,3)))
    model.add(AveragePooling2D())
    model.add(Dropout(0.5))
    
    model.add(Flatten())

    model.add(Dense(32, activation='relu',kernel_regularizer = L1L2(l1=0.01, l2=0.01) , bias_regularizer = L1L2(l1=0.01, l2=0.01) ))
    model.add(Dropout(0.5))
    
    model.add(Dense(3, activation='softmax',kernel_regularizer = L1L2(l1=0.01, l2=0.01) , bias_regularizer = L1L2(l1=0.01, l2=0.01))) # Last fully-connected layer of 3 outputs (3 categories).
    model.summary()
    
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(5e-4), metrics=['accuracy'])

    def scheduler(epoch, lr):
        if epoch < 4 :
            return 5e-4
        if epoch < 10 :
            return lr * tf.math.exp(-0.05)
        if epoch < 20 :
            return lr * tf.math.exp(-0.5)

    Model_Checkpoint = tf.keras.callbacks.ModelCheckpoint(f"XCeption_April4th_1",save_best_only=True)
    
    
    Learningrate_Scheduler  = tf.keras.callbacks.LearningRateScheduler(scheduler)
    
    
    EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=5, mode='auto')
    
    model.fit(  X_train, 
                y_train,
                validation_split = .3, 
                epochs = 20,
                callbacks=[Model_Checkpoint,Learningrate_Scheduler,EarlyStopping])
    

    model.save(f"XCeption_April4th_1.h5")


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 4, 4, 2048)        20861480  
                                                                 
 average_pooling2d_3 (Averag  (None, 2, 2, 2048)       0         
 ePooling2D)                                                     
                                                                 
 dropout_6 (Dropout)         (None, 2, 2, 2048)        0         
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 32)                262176    
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                      

INFO:tensorflow:Assets written to: XCeption_April4th_1/assets


INFO:tensorflow:Assets written to: XCeption_April4th_1/assets


329/329 [==============================] - 175s 516ms/step - loss: 3.6564 - accuracy: 0.8226 - val_loss: 1.0262 - val_accuracy: 0.8438 - lr: 5.0000e-04
Epoch 2/20
329/329 [==============================] - ETA: 0s - loss: 0.9135 - accuracy: 0.8942

INFO:tensorflow:Assets written to: XCeption_April4th_1/assets


...

INFO:tensorflow:Assets written to: XCeption_April4th_1/assets


329/329 [==============================] - 167s 508ms/step - loss: 0.2655 - accuracy: 0.9951 - val_loss: 0.4797 - val_accuracy: 0.9211 - lr: 4.1149e-06
Epoch 20/20
329/329 [==============================] - ETA: 0s - loss: 0.2625 - accuracy: 0.9957

INFO:tensorflow:Assets written to: XCeption_April4th_1/assets


INFO:tensorflow:Assets written to: XCeption_April4th_1/assets


329/329 [==============================] - 167s 507ms/step - loss: 0.2625 - accuracy: 0.9957 - val_loss: 0.4778 - val_accuracy: 0.9224 - lr: 2.4958e-06


#### Save preprocessor function to local "preprocessor.zip" file

In [75]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

cannot pickle 'module' object
Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [76]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=True,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

/home/ecbm4040/climate_change/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/lib/python3.8/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-04-05 00:46:40,881 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-04-05 00:46:48,420 - INFO - Signatures found in model: [serving_default].
2022-04-05 00:46:48,420 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-04-05 00:46:48,421 - INFO - Output names: ['dense_7']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-04-05 00:46:51,293 - WARNING - From /home/ecbm4040/climate_change

## 4. Generate predictions from X_test data and submit model to competition


In [77]:
# import and preprocess X_test images in correct order...
# ...for leaderboard prediction submissions
filenumbers = [str(x) for x in range(1, 5001)]
filenames = ["competition_data/testdata/test/test"+x for x in filenumbers]

# preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data = []
for i in filenames:
    try:
        preprocessed_image_data.append(preprocessor(i))
    except:
        pass

In [78]:
X_test=np.vstack(preprocessed_image_data) 


In [80]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

# Note -- This is the unique rest api that powers this climate change image classification  Model Plaground
# ... Update the apiurl if submitting to a new competition

apiurl="https://srdmat3yhf.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

AI Modelshare Username:········
AI Modelshare Password:········
AI Model Share login credentials set successfully.


In [81]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [82]:
decoding_list = ["forest","nonforest","snow_shadow_cloud"] 

In [83]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [decoding_list[i] for i in prediction_column_index]

157/157 [==============================] - 16s 99ms/step


In [84]:
pandas.Series(prediction_column_index).value_counts()

1    2338
0    1890
2     772
dtype: int64

In [85]:
# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

  7% [...                                                 ]  1843200 / 24195744

 47% [........................                            ] 11591680 / 24195744

 88% [..............................................      ] 21479424 / 24195744

100% [....................................................] 24195744 / 24195744

/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:287: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  leaderboard = leaderboard.append(metadata, ignore_index=True, sort=False)
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:289: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  leaderboard['username']=leaderboard.pop("username")
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:290: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

  9% [....                                                ]  1941504 / 20976109

 57% [.............................                       ] 12083200 / 20976109

100% [....................................................] 20976109 / 20976109

/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:287: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  leaderboard = leaderboard.append(metadata, ignore_index=True, sort=False)
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:289: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  leaderboard['username']=leaderboard.pop("username")
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:290: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 376

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1535


In [86]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

In [87]:
data[data["username"] == "vkalmath"]

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,...,Deep Learning,Optimizer,Transfer Learning,member1,member2,MEMBER1,MEMBER2,username,timestamp,version
14,0.8000,0.764359,0.815173,0.780444,keras,True,True,Sequential,8.0,135491.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-31 02:00:13.166990,315
20,0.7648,0.776539,0.791213,0.827111,keras,NaN,True,Sequential,8.0,136387.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-19 19:11:33.485952,88
22,0.7948,0.754853,0.813228,0.776667,keras,True,True,Sequential,9.0,529603.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-30 02:11:01.966997,279
23,0.7844,0.751097,0.817057,0.778444,keras,True,True,Sequential,13.0,22173187.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-27 03:40:38.054515,217
24,0.7936,0.749585,0.816209,0.770667,keras,True,True,Sequential,9.0,529603.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-30 15:02:38.506152,291
30,0.7600,0.750484,0.798412,0.787111,keras,NaN,True,Functional,6.0,4721667.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-25 03:05:21.678937,170
39,0.7808,0.731753,0.810912,0.759111,keras,True,True,Sequential,3.0,6147.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-30 16:41:01.577642,295
53,0.7740,0.732417,0.788778,0.749556,keras,True,True,Sequential,11.0,529603.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-30 01:10:44.756222,278
67,0.7652,0.714668,0.799811,0.733111,keras,True,True,Sequential,6.0,65795.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-30 19:11:09.536784,299
68,0.7528,0.715706,0.797182,0.742667,keras,True,True,Sequential,13.0,22173187.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vkalmath,2022-03-27 16:56:23.923381,228


## 5. Repeat submission process to improve place on leaderboard

*Train and submit your own models using code modeled after what you see above.*

It may also be useful to examine the architeture of models that perform particuarly well/poorly, or to compare models you've created with similar models submitted by others. Use the compare_models function in combination with the leaderboard to learn more about models that been previously submitted and potentially make decisiona about what you should do next.

In [ ]:
# Compare two or more models
data=mycompetition.compare_models([1, 5], verbose=1)
mycompetition.stylize_compare(data)